In [1]:
import nest_asyncio
nest_asyncio.apply()

from hddCRP.modelBuilder import cdCRP
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from statannotations.Annotator import Annotator

import os

In [2]:
session_numbers = [1]; # index by 1

overwrite_existing_results = True
results_directory = "Results/individualFit/"

if(not os.path.exists(results_directory)):
    os.makedirs(results_directory)

context_depth = 1;
nback_depth = 1;

data_filename = 'data/Data_turns_all_by_session.pkl';
with open(data_filename, 'rb') as data_file:
    data = pickle.load(data_file)

subjects = list(data["data"].keys())
subjects.sort()
print("subjects = " + str(subjects))

action_labels = [0,1,2]

subjects = ['A1', 'A2', 'B1', 'B2', 'C1', 'C2', 'D1', 'D2', 'E1', 'E2', 'F1', 'F2', 'G1', 'G2', 'H1', 'I1', 'I2', 'J1', 'J2']


In [3]:
start_session = np.min(session_numbers)
end_session = np.max(session_numbers)
fit_file = f"{results_directory}/fits_session_{start_session}"
fit_summary_file = f"{results_directory}/fit_summary_session_{start_session}"
if(end_session != start_session):
    fit_file += f"_to_{start_session}"
    fit_summary_file  += f"_to_{start_session}"
    
if(nback_depth != 1 or context_depth != 2):
    fit_file += f"_cd{context_depth}_nb{nback_depth}"
    fit_summary_file  += f"_cd{context_depth}_nb{nback_depth}"

fit_file += f".pkl"
fit_summary_file += f".pkl"

if(not os.path.isfile(fit_file) or overwrite_existing_results):
    data_fits = pd.DataFrame()
    data_fit_metrics = pd.DataFrame()
    for subject_index, subject in enumerate(subjects):
        print(f"subject {subject} ")
        sequences_0 = data["data"][subject]["data"]; # turns in each session
        session_types_0 = data["data"][subject]["task"] # which maze

        ii = list(np.where(np.array(session_types_0)=='C')[0][np.array(session_numbers)-1]) # sessions in map C
        sequences     = [sequences_0[xx] for xx in ii]
        session_types = [session_types_0[xx] for xx in ii]

        stan_seed = (subject_index+1) * 1000 + start_session

        # build model with given sequences
        model = cdCRP(sequences, session_labels=session_types, subject_labels=subject, possible_observations=action_labels);

        # set model depth
        model.same_nback_depth = nback_depth
        model.context_depth = context_depth

        raise RuntimeError

        # fit with Stan
        model.build(random_seed=stan_seed);
        model.fit_model()

        map_fit = model.get_map()
        fit_df  = model.fit.to_frame()
        summary_df = model.fit_summary()

        fit_df["subject"] = subject
        fit_df["start_session_C"] = start_session
        fit_df["end_session_C"] = end_session
        summary_df["subject"] = subject
        summary_df["MAP"] = pd.Series(map_fit)
        summary_df["start_session_C"] = start_session
        summary_df["end_session_C"] = end_session

        data_fit_metrics = pd.concat([data_fit_metrics,summary_df], copy=False)
        data_fits = pd.concat([data_fits,fit_df], copy=False)

    data_fits.to_pickle(fit_file)
    data_fit_metrics.to_pickle(fit_summary_file)
else:
    print("fit file found")



subject A1 


RuntimeError: 

In [4]:
# print(model.model)
model.build(random_seed=stan_seed);
model.fit_model()
    

Building...

/home/latimerk/.cache/httpstan/4.10.1/models/draa4kh3/model_draa4kh3.cpp: In constructor ‘model_draa4kh3_namespace::model_draa4kh3::model_draa4kh3(stan::io::var_context&, unsigned int, std::ostream*)’:
/home/latimerk/.cache/httpstan/4.10.1/models/draa4kh3/model_draa4kh3.cpp:608:17: warning: variable ‘bb_c’ set but not used [-Wunused-but-set-variable]
             int bb_c = std::numeric_limits<int>::min();
                 ^~~~
In file included from /home/latimerk/anaconda3/envs/JaiYuLab/lib/python3.10/site-packages/httpstan/include/stan/math/prim/fun.hpp:124:0,
                 from /home/latimerk/anaconda3/envs/JaiYuLab/lib/python3.10/site-packages/httpstan/include/stan/math/rev/fun/multiply.hpp:7,
                 from /home/latimerk/anaconda3/envs/JaiYuLab/lib/python3.10/site-packages/httpstan/include/stan/math/rev/fun/elt_multiply.hpp:9,
                 from /home/latimerk/anaconda3/envs/JaiYuLab/lib/python3.10/site-packages/httpstan/include/stan/math/rev/fun.hpp:55,
            


Building: 35.2s, done.Messages from stanc:
    means either no prior is provided, or the prior(s) depend on data
    variables. In the later case, this may be a false positive.
    prior is provided, or the prior(s) depend on data variables. In the later
    case, this may be a false positive.
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   1% (103/8000)
Sampling:   3% (202/8000)
Sampling:   4% (302/8000)
Sampling:   6% (501/8000)
Sampling:   8% (600/8000)
Sampling:  10% (800/8000)
Sampling:  11% (900/8000)
Sampling:  12% (1000/8000)
Sampling:  15% (1200/8000)
Sampling:  16% (1300/8000)
Sampling:  19% (1500/8000)
Sampling:  21% (1700/8000)
Sampling:  24% (1900/8000)
Sampling:  2

<stan.Fit>
Parameters:
    alpha: ()
    timeconstant_within_session_C: ()
    repeat_bias_1_back: ()
    context_similarity_depth_1: ()
    log_likelihood: (1,)
Draws: 4000

/home/latimerk/anaconda3/envs/JaiYuLab/lib/python3.10/site-packages/arviz/stats/stats.py:1673: UserWarning: The point-wise WAIC is the same with the sum WAIC, please double check
            the Observed RV in your model to make sure it returns element-wise logp.
            
  warnings.warn(


Computed from 4000 posterior samples and 1 observations log-likelihood matrix.

          Estimate       SE
elpd_waic   -55.00     0.00
p_waic        0.26        -

In [ ]:
fit_summary = pd.read_pickle(fit_summary_file)
group_names = np.array(["uniform", "diverse"])

mapping = {}
for jj in range(2):
    for ii in data["group_definition"][group_names[jj]]:
        mapping[ii] = group_names[jj]
fit_summary["group"] = fit_summary["subject"].map(mapping)

params = list(fit_summary.index.unique())

fit_summary.index.name = 'parameter'
fit_summary = fit_summary.reset_index()

estimator_to_plot = "median"

formatted_fit = fit_summary.pivot(index="subject", columns=["parameter"],values=[estimator_to_plot]).droplevel(0,axis=1)
formatted_fit["group"] = fit_summary[["group","subject"]].drop_duplicates("subject").set_index("subject")

#     print(fit_summary["median"][fit_summary["parameter"] == ii])
plt.figure(figsize=(10,6))
g = sns.PairGrid(formatted_fit, hue="group")
g.map_upper(sns.scatterplot)
g.map_diag(sns.histplot)
g.add_legend()
# fit_summary[["median","subject"]].set_index("subject")

In [ ]:

grps = [["timeconstant_within_session_C"], ["alpha"], ["repeat_bias_1_back", "context_similarity_depth_1", "context_similarity_depth_2"]]
ii = 0
plt.figure(figsize=(len(grps[ii])*3,4))
fit_summary2 = fit_summary[["group", estimator_to_plot, "parameter"]]
ax = sns.boxplot(data=fit_summary2, x="parameter", y=estimator_to_plot, order=grps[ii], hue_order=["uniform", "diverse"],
            hue="group")

pairs = [((xx, "uniform"), (xx, "diverse")) for xx in grps[ii]]
annot = Annotator(ax, pairs=pairs, data=fit_summary2, x="parameter", y=estimator_to_plot, order=grps[ii], hue="group", hue_order=["uniform", "diverse"])
#Annotator.new_plot(ax=ax, pairs=pairs, data=fit_summary2, x="parameter", y=estimator_to_plot, order=grps[ii], hue="group", hue_order=["uniform", "diverse"])
annot.configure(test='Mann-Whitney', verbose=2, text_format='simple')
annot.apply_test()
annot.annotate()
sp = sns.stripplot(data=fit_summary2,  x="parameter", y=estimator_to_plot, hue="group", order=grps[ii], hue_order=["uniform", "diverse"],
              dodge=True, size=6, alpha=1,  edgecolor=[0,0,0], linewidth=0.5)
sns.despine(offset=5, trim=True);
plt.xlabel(None);


In [ ]:
ii = 1
plt.figure(figsize=(len(grps[ii])*3,4))
ax = sns.boxplot(x="parameter", y=estimator_to_plot, order=grps[ii], hue_order=["uniform", "diverse"],
            hue="group",
            data=fit_summary2)

pairs = [((xx, "uniform"), (xx, "diverse")) for xx in grps[ii]]
annot = Annotator(ax, pairs=pairs, data=fit_summary2, x="parameter", y=estimator_to_plot, order=grps[ii], hue="group", hue_order=["uniform", "diverse"])
annot.configure(test='Mann-Whitney', verbose=2, text_format='simple')

annot.apply_test()
annot.annotate()

sns.stripplot(data=fit_summary2, x="parameter", y=estimator_to_plot, hue="group", size=6, order=grps[ii], hue_order=["uniform", "diverse"],
             dodge=True, alpha=1,  edgecolor=[0,0,0], linewidth=0.5)
sns.despine(offset=5, trim=True)
plt.xlabel(None)

In [ ]:
ii = 2
plt.figure(figsize=(len(grps[ii])*3,4))
ax = sns.boxplot(x="parameter", y=estimator_to_plot, order=grps[ii], hue_order=["uniform", "diverse"],
            hue="group",
            data=fit_summary2)

pairs = [((xx, "uniform"), (xx, "diverse")) for xx in grps[ii]]
annot = Annotator(ax, pairs=pairs, data=fit_summary2, x="parameter", y=estimator_to_plot, order=grps[ii], hue="group", hue_order=["uniform", "diverse"])
#Annotator.new_plot(ax=ax, pairs=pairs, data=fit_summary2, x="parameter", y=estimator_to_plot, order=grps[ii], hue="group", hue_order=["uniform", "diverse"])
annot.configure(test='Mann-Whitney', verbose=2, text_format='simple')
annot.apply_test()
annot.annotate()
sns.stripplot(data=fit_summary2, x="parameter", y=estimator_to_plot, hue="group", size=6, order=grps[ii], hue_order=["uniform", "diverse"],
             dodge=True, alpha=1,  edgecolor=[0,0,0], linewidth=0.5)
sns.despine(offset=5, trim=True)
plt.xlabel(None)

In [ ]:
fit_summary.head()

In [ ]:
x = fit_summary.pivot(index="subject", columns=["parameter"],values=[estimator_to_plot]).droplevel(0,axis=1)
x["group"] = fit_summary[["group","subject"]].drop_duplicates("subject").set_index("subject")
# fit_summary.pivot(index="subject", columns=["parameter"], values=["group"])
x